# Noisy circuits [[Link]](https://pennylane.ai/qml/demos/tutorial_noisy_circuits.html)


Noise can be separated into two categories.
* **Coherent noise** is described by unitary operations that maintain the purity of the output quantum state. <br>
A common source are systematic errors originating from imperfectly-calibrated devices that do not exactly apply the desired gates, e.g., applying a rotation by an angle $\phi+\epsilon$ instead of $\phi$.
* **Incoherent noise** is more problematic: it originates from a quantum computer becoming entangled with the environment, resulting in mixed states -- probability distributions over different pure states. <br>
Incoherent noise thus leads to outputs that are always random, regardless of what basis we measure in.

The purpose of PennyLane's `default.mixed` device is to provide native support for mixed states and for simulating noisy computations. 

Let's use `default.mixed` to simulate a simple circuit for preparing the Bell state $|\psi\rangle =\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$. <br>
We ask the QNode to return the expectation value of $Z_0\otimes Z_1$:

In [2]:
import pennylane as qml
from pennylane import numpy as np

In [9]:
dev = qml.device('default.mixed', wires=2)
print(f"Input state is  = \n{np.real(dev.state)}")

Input state is  = 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [10]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


print(f"QNode output = {circuit():.4f}")

QNode output = 1.0000


The device stores the output state as a density matrix. In this case, the density matrix is equal to $|\psi\rangle\langle\psi|$, where $|\psi\rangle=\frac{1}{\sqrt{2}}|00\rangle+|11\rangle$.

In [11]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]


Incoherent noise is modelled by quantum channels.

Mathematically, a quantum channel is a linear, completely positve, and trace-preserving (CPTP) map. <br>
A convenient strategy for representing quantum channnels is to empoly Kraus operators $\{K_i\}$ satisfying the condition $\sum_iK_i^\dagger K_i=I$. <br>
For an initial state $\rho$, the output state after the action of a channel $\Phi$ is:
$$
\Phi(\rho) = \sum_i K_i\rho K_i^\dagger.
$$